In [ ]:
##Adapt the code from class that reads GHCN Daily Data from Amazon Web Services
#and write a function that will download the station you want (called with a GHCN station ID), 
#and calculate the all time record high and low and the normal (mean) high and low temperature for the 1981-2010 period for the desired station and returns a pandas data frame 
#with the columns ['record_min_temp', average_min_temp', 'average_max_temp', record_max_temp'].  
#Write a code that can call this function and successfully demonstrate that it works.



###Final Function at End of Code

In [9]:
import pandas as pd

In [10]:
# Choose Station ID  (Dubai ITL) AEM00041194
stn = 'AEM00041194'

#1981 - 2010
time_start = '1981-01-01'
time_end = '2010-01-01'


In [11]:


df = (
    pd.read_csv(
        "s3://noaa-ghcn-pds/csv/by_station/" + stn + ".csv",
        storage_options={"anon": True},  # Anonymous access to S3
        dtype={'Q_FLAG': 'object', 'M_FLAG': 'object'},  # Define data types
        parse_dates=['DATE']  # Parse the 'DATE' column as datetime
    )
    .set_index('DATE')  # Set 'DATE' as the DataFrame index
    .sort_index()  
)

df

,ID,ELEMENT,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME
DATE,,,,,,,
1983-01-01,AEM00041194,TMAX,276,NaN,NaN,S,NaN
1983-01-01,AEM00041194,TAVG,195,H,NaN,S,NaN
1983-01-01,AEM00041194,TMIN,140,NaN,NaN,S,NaN
1983-01-02,AEM00041194,TMIN,134,NaN,NaN,S,NaN
1983-01-02,AEM00041194,TAVG,198,H,NaN,S,NaN
...,...,...,...,...,...,...,...
2024-09-11,AEM00041194,TAVG,366,H,NaN,S,NaN
2024-09-12,AEM00041194,TMIN,314,NaN,NaN,S,NaN
2024-09-12,AEM00041194,TMAX,452,NaN,NaN,S,NaN


In [12]:
df_range = df.loc[time_start:time_end]

df_range

,ID,ELEMENT,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME
DATE,,,,,,,
1983-01-01,AEM00041194,TMAX,276,NaN,NaN,S,NaN
1983-01-01,AEM00041194,TAVG,195,H,NaN,S,NaN
1983-01-01,AEM00041194,TMIN,140,NaN,NaN,S,NaN
1983-01-02,AEM00041194,TMIN,134,NaN,NaN,S,NaN
1983-01-02,AEM00041194,TAVG,198,H,NaN,S,NaN
...,...,...,...,...,...,...,...
2009-12-31,AEM00041194,PRCP,0,NaN,NaN,S,NaN
2010-01-01,AEM00041194,PRCP,0,NaN,NaN,S,NaN
2010-01-01,AEM00041194,TMIN,168,NaN,NaN,S,NaN


In [39]:
df_tmax = df_range.loc[df_range['ELEMENT'] == 'TMAX', 'DATA_VALUE'] /10
df_tmin = df_range.loc[df_range['ELEMENT'] == 'TMIN', 'DATA_VALUE'] /10


In [40]:
#annual mean min temp
annual_mean_min=df_tmin[~((df_tmin.index.month==2)&(df_tmin.index.day==29))]

#annual mean max temp
annual_mean_max=df_tmax[~((df_tmax.index.month==2)&(df_tmax.index.day==29))]

In [41]:
df_elements = pd.DataFrame({
    'record_min_temp':[df_tmin],
    'average_min_temp':[annual_mean_min],
    'average_max_temp':[annual_mean_max],
    'record_max_temp':[df_tmax]})

In [63]:
def station_data_values(station_id, time_start, time_end):
    #reads in data
    df = (
    pd.read_csv(
        "s3://noaa-ghcn-pds/csv/by_station/" + station_id + ".csv",
        storage_options={"anon": True},  # Anonymous access to S3
        dtype={'Q_FLAG': 'object', 'M_FLAG': 'object'},  # Define data types
        parse_dates=['DATE']  # Parse the 'DATE' column as datetime
    )
    .set_index('DATE')  # Set 'DATE' as the DataFrame index
    .sort_index()  
)
    df_range = df.loc[time_start:time_end]

     #find max tmax and min tmin
    df_tmax = df_range.loc[df_range['ELEMENT'] == 'TMAX', 'DATA_VALUE'] /10
    df_tmin = df_range.loc[df_range['ELEMENT'] == 'TMIN', 'DATA_VALUE'] /10
   
    #annual mean min temp
    ser=df_tmin[~((df_tmin.index.month==2)&(df_tmin.index.day==29))]
    tmin_mean =ser.groupby(ser.index.day_of_year).mean()

    #annual mean max temp
    ser2=df_tmax[~((df_tmax.index.month==2)&(df_tmax.index.day==29))]
    tmax_mean=ser2.groupby(ser2.index.day_of_year).mean()

    #record min
    record_tmin = ser.groupby(ser.index.day_of_year).min()
    #record max
    record_tmax = ser2.groupby(ser2.index.day_of_year).max()  
    
    #create pandas df of all values

    df_elements_daily = pd.concat(
        {'record_min_temp': record_tmin, 'average_min_temp': tmin_mean, 'average_max_temp': tmax_mean,'record_max_temp':record_tmax},
        axis=1
    )
    return df_elements_daily
    


In [67]:

station_data_values('AEM00041194','1981-01-01', '2010-01-01')

,record_min_temp,average_min_temp,average_max_temp,record_max_temp
DATE,,,,
1,10.0,14.147619,24.518182,28.2
2,12.1,14.794737,25.024000,30.2
3,11.2,14.316667,25.432000,29.4
4,10.0,14.435714,24.780000,28.9
5,11.7,14.733333,24.471429,28.2
...,...,...,...,...
362,8.2,14.976190,24.673913,29.5
363,9.7,15.194737,24.600000,27.7
364,9.8,14.959091,24.258333,27.0
